# 🚀 Advanced Topics & Production Deployment

## 🎯 Learning Objectives
- **Regularization**: Ridge, Lasso, and Elastic Net regression
- **Model Drift Detection**: Monitoring model performance over time
- **Hyperparameter Tuning**: Optimizing model parameters
- **AWS Deployment**: Production-ready model deployment

## 🏢 Business Context: Production-Ready ML

As Amazon's senior data scientist, you need to:
- Deploy models that scale with business growth
- Monitor performance and detect when models need retraining
- Optimize models for both accuracy and interpretability
- Ensure models work reliably in production environments

In [ ]:
# Import advanced libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
import joblib
import pickle
import json
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

print("✅ Advanced libraries imported!")

## 🛡️ Regularization Techniques

### **Why Regularization?**
- Prevents overfitting by adding penalty terms
- Improves model generalization
- Handles multicollinearity

### **Types of Regularization:**
1. **Ridge (L2)**: Penalizes large coefficients
2. **Lasso (L1)**: Encourages sparse models (feature selection)
3. **Elastic Net**: Combines both L1 and L2 penalties

In [ ]:
# Generate sample data for regularization demonstration
np.random.seed(42)
n_samples, n_features = 1000, 20

# Create features with some multicollinearity
X = np.random.randn(n_samples, n_features)
# Add some correlated features
X[:, 5] = X[:, 0] * 0.8 + np.random.normal(0, 0.1, n_samples)
X[:, 10] = X[:, 1] * 0.9 + np.random.normal(0, 0.1, n_samples)

# Create target with some noise
true_coefficients = np.array([2.0, -1.5, 1.0, 0.8, -0.5] + [0.1] * 15)
y = X @ true_coefficients + np.random.normal(0, 0.5, n_samples)

# Split data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"📊 Dataset: {n_samples} samples, {n_features} features")
print(f"🎯 Target range: {y.min():.2f} to {y.max():.2f}")

In [ ]:
# Compare different regularization techniques
models = {
    'Linear Regression': LinearRegression(),
    'Ridge (α=1.0)': Ridge(alpha=1.0),
    'Lasso (α=0.1)': Lasso(alpha=0.1),
    'Elastic Net (α=0.1, l1_ratio=0.5)': ElasticNet(alpha=0.1, l1_ratio=0.5)
}

results = {}
for name, model in models.items():
    model.fit(X_train_scaled, y_train)
    y_pred = model.predict(X_test_scaled)
    
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    # Count non-zero coefficients (for Lasso/Elastic Net)
    non_zero_coeffs = np.sum(model.coef_ != 0) if hasattr(model, 'coef_') else n_features
    
    results[name] = {
        'MSE': mse,
        'R²': r2,
        'Non-zero coefficients': non_zero_coeffs
    }

# Display results
results_df = pd.DataFrame(results).T
print("📊 Regularization Comparison:")
print("=" * 50)
print(results_df.round(4))

# Visualize coefficient differences
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
fig.suptitle('Coefficient Comparison Across Regularization Methods', fontsize=16)

for i, (name, model) in enumerate(models.items()):
    row, col = i // 2, i % 2
    axes[row, col].bar(range(len(model.coef_)), model.coef_, alpha=0.7)
    axes[row, col].set_title(name)
    axes[row, col].set_xlabel('Feature Index')
    axes[row, col].set_ylabel('Coefficient Value')
    axes[row, col].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 🎛️ Hyperparameter Tuning

### **Grid Search vs Random Search**
- **Grid Search**: Exhaustive search over parameter grid
- **Random Search**: Random sampling from parameter space
- **Business Impact**: Better models = better predictions = more revenue

In [ ]:
# Hyperparameter tuning with Grid Search
param_grid = {
    'alpha': [0.001, 0.01, 0.1, 1.0, 10.0, 100.0],
    'l1_ratio': [0.1, 0.3, 0.5, 0.7, 0.9]
}

elastic_net = ElasticNet(random_state=42, max_iter=1000)
grid_search = GridSearchCV(
    elastic_net, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1
)
grid_search.fit(X_train_scaled, y_train)

print("🎛️ Best Hyperparameters:")
print(f"Alpha: {grid_search.best_params_['alpha']}")
print(f"L1 Ratio: {grid_search.best_params_['l1_ratio']}")
print(f"Best CV Score (RMSE): {np.sqrt(-grid_search.best_score_):.4f}")

# Compare with default parameters
default_model = ElasticNet(random_state=42)
default_model.fit(X_train_scaled, y_train)
default_score = np.sqrt(mean_squared_error(y_test, default_model.predict(X_test_scaled)))

tuned_model = grid_search.best_estimator_
tuned_score = np.sqrt(mean_squared_error(y_test, tuned_model.predict(X_test_scaled)))

improvement = ((default_score - tuned_score) / default_score) * 100
print(f"\n📈 Improvement: {improvement:.2f}% better RMSE with tuning")

## 📉 Model Drift Detection

### **What is Model Drift?**
- Model performance degrades over time
- Data distribution changes
- Business environment evolves

### **Detection Methods:**
1. **Performance Monitoring**: Track metrics over time
2. **Data Drift**: Monitor feature distributions
3. **Concept Drift**: Monitor target variable patterns

In [ ]:
# Simulate model drift over time
np.random.seed(42)
n_days = 100
dates = pd.date_range('2023-01-01', periods=n_days, freq='D')

# Generate data with drift
base_performance = 0.85  # R² score
drift_factor = 0.001     # Daily degradation
noise = 0.02            # Random variation

performance_over_time = []
for day in range(n_days):
    # Simulate performance degradation with some recovery
    drift = base_performance - (day * drift_factor) + np.random.normal(0, noise)
    performance_over_time.append(max(0.5, drift))  # Don't go below 0.5

# Create drift detection function
def detect_drift(performance_history, window=7, threshold=0.05):
    """Detect significant performance degradation"""
    if len(performance_history) < window * 2:
        return False, 0
    
    recent_avg = np.mean(performance_history[-window:])
    baseline_avg = np.mean(performance_history[-2*window:-window])
    
    degradation = (baseline_avg - recent_avg) / baseline_avg
    return degradation > threshold, degradation

# Simulate drift detection
drift_detected = []
degradation_levels = []

for i in range(14, len(performance_over_time)):
    detected, degradation = detect_drift(performance_over_time[:i+1])
    drift_detected.append(detected)
    degradation_levels.append(degradation)

# Visualize drift
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8))

# Performance over time
ax1.plot(dates[14:], performance_over_time[14:], 'b-', label='Model Performance (R²)')
ax1.axhline(y=0.8, color='red', linestyle='--', alpha=0.7, label='Acceptable Threshold')
ax1.set_ylabel('R² Score')
ax1.set_title('Model Performance Over Time')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Drift detection
drift_dates = [dates[i+14] for i, detected in enumerate(drift_detected) if detected]
ax2.plot(dates[14:], degradation_levels, 'g-', label='Degradation Level')
ax2.axhline(y=0.05, color='red', linestyle='--', alpha=0.7, label='Drift Threshold')
if drift_dates:
    ax2.scatter(drift_dates, [0.06] * len(drift_dates), color='red', s=100, 
                marker='^', label='Drift Detected')
ax2.set_ylabel('Degradation Level')
ax2.set_xlabel('Date')
ax2.set_title('Model Drift Detection')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"📉 Drift Detection Results:")
print(f"• Drift detected on {len(drift_dates)} occasions")
print(f"• Average degradation: {np.mean(degradation_levels):.3f}")
print(f"• Final performance: {performance_over_time[-1]:.3f}")

## 🚀 AWS Deployment Guide

### **Step 1: Model Serialization**
Save your trained model for deployment

In [ ]:
# Model serialization
def save_model_pipeline(model, scaler, feature_names, model_name='amazon_sales_model'):
    """Save model and preprocessing pipeline"""
    
    # Create model artifacts
    model_artifacts = {
        'model': model,
        'scaler': scaler,
        'feature_names': feature_names,
        'model_info': {
            'created_date': datetime.now().isoformat(),
            'model_type': type(model).__name__,
            'version': '1.0.0'
        }
    }
    
    # Save using joblib (better for scikit-learn models)
    joblib.dump(model_artifacts, f'{model_name}.joblib')
    
    # Also save as pickle for compatibility
    with open(f'{model_name}.pkl', 'wb') as f:
        pickle.dump(model_artifacts, f)
    
    print(f"✅ Model saved as {model_name}.joblib and {model_name}.pkl")
    return model_artifacts

# Example usage
feature_names = [f'feature_{i}' for i in range(X_train.shape[1])]
model_artifacts = save_model_pipeline(tuned_model, scaler, feature_names)

# Create prediction function
def predict_sales(input_data, model_artifacts):
    """Make predictions using saved model"""
    model = model_artifacts['model']
    scaler = model_artifacts['scaler']
    
    # Preprocess input
    input_scaled = scaler.transform(input_data)
    
    # Make prediction
    prediction = model.predict(input_scaled)
    
    return prediction

# Test prediction
test_input = np.random.randn(1, X_train.shape[1])
prediction = predict_sales(test_input, model_artifacts)
print(f"\n🧪 Test Prediction: {prediction[0]:.2f}")

### **Step 2: AWS Lambda Function**
Create a serverless prediction API

In [ ]:
# Create Lambda function code
lambda_code = '''
import json
import joblib
import numpy as np
from datetime import datetime

def lambda_handler(event, context):
    """AWS Lambda function for sales prediction"""
    
    try:
        # Load model artifacts
        model_artifacts = joblib.load('/opt/model/amazon_sales_model.joblib')
        
        # Parse input
        body = json.loads(event['body'])
        input_data = np.array(body['features']).reshape(1, -1)
        
        # Make prediction
        prediction = model_artifacts['model'].predict(
            model_artifacts['scaler'].transform(input_data)
        )[0]
        
        # Prepare response
        response = {
            'prediction': float(prediction),
            'timestamp': datetime.now().isoformat(),
            'model_version': model_artifacts['model_info']['version']
        }
        
        return {
            'statusCode': 200,
            'headers': {'Content-Type': 'application/json'},
            'body': json.dumps(response)
        }
        
    except Exception as e:
        return {
            'statusCode': 500,
            'headers': {'Content-Type': 'application/json'},
            'body': json.dumps({'error': str(e)})
        }
'''

# Save Lambda function
with open('lambda_function.py', 'w') as f:
    f.write(lambda_code)

print("✅ Lambda function created: lambda_function.py")
print("\n📋 AWS Deployment Steps:")
print("1. Upload model artifacts to S3")
print("2. Create Lambda function with the code above")
print("3. Configure API Gateway for HTTP endpoints")
print("4. Set up CloudWatch for monitoring")

### **Step 3: AWS Deployment Commands**

```bash
# 1. Install AWS CLI and configure credentials
aws configure

# 2. Create S3 bucket for model artifacts
aws s3 mb s3://amazon-sales-ml-models

# 3. Upload model to S3
aws s3 cp amazon_sales_model.joblib s3://amazon-sales-ml-models/

# 4. Create Lambda deployment package
pip install -r requirements.txt -t lambda_package/
cp lambda_function.py lambda_package/
cd lambda_package && zip -r ../lambda_deployment.zip .

# 5. Create Lambda function
aws lambda create-function \
    --function-name amazon-sales-predictor \
    --runtime python3.9 \
    --role arn:aws:iam::YOUR_ACCOUNT:role/lambda-execution-role \
    --handler lambda_function.lambda_handler \
    --zip-file fileb://lambda_deployment.zip \
    --timeout 30 \
    --memory-size 512

# 6. Create API Gateway
aws apigateway create-rest-api --name "Sales Prediction API"
```

### **Step 4: Docker Alternative (ECS/Fargate)**

```dockerfile
# Dockerfile for containerized deployment
FROM python:3.9-slim

WORKDIR /app
COPY requirements.txt .
RUN pip install -r requirements.txt

COPY model/ ./model/
COPY app.py .

EXPOSE 8080
CMD ["python", "app.py"]
```

In [ ]:
# Create Flask app for containerized deployment
flask_app_code = '''
from flask import Flask, request, jsonify
import joblib
import numpy as np
from datetime import datetime

app = Flask(__name__)

# Load model
model_artifacts = joblib.load('model/amazon_sales_model.joblib')

@app.route('/predict', methods=['POST'])
def predict():
    try:
        data = request.get_json()
        features = np.array(data['features']).reshape(1, -1)
        
        prediction = model_artifacts['model'].predict(
            model_artifacts['scaler'].transform(features)
        )[0]
        
        return jsonify({
            'prediction': float(prediction),
            'timestamp': datetime.now().isoformat(),
            'model_version': model_artifacts['model_info']['version']
        })
    except Exception as e:
        return jsonify({'error': str(e)}), 500

@app.route('/health', methods=['GET'])
def health():
    return jsonify({'status': 'healthy'})

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=8080)
'''

with open('app.py', 'w') as f:
    f.write(flask_app_code)

print("✅ Flask app created: app.py")
print("\n🐳 Docker Commands:")
print("docker build -t amazon-sales-predictor .")
print("docker run -p 8080:8080 amazon-sales-predictor")

## 📊 Production Monitoring

### **Key Metrics to Monitor:**
1. **Model Performance**: Prediction accuracy over time
2. **System Performance**: Response time, throughput
3. **Business Impact**: Revenue impact of predictions
4. **Data Quality**: Input data validation

### **AWS Services for Monitoring:**
- **CloudWatch**: Metrics, logs, alarms
- **X-Ray**: Distributed tracing
- **SageMaker Model Monitor**: Automated drift detection
- **CloudTrail**: API call logging

## 🎯 Business Impact & ROI

### **Expected Benefits:**
- **5-15% Revenue Increase**: Better demand forecasting
- **10-20% Cost Reduction**: Optimized inventory management
- **Improved Customer Satisfaction**: Better product availability
- **Data-Driven Decisions**: Reduced guesswork in strategy

### **Success Metrics:**
- Model accuracy > 85%
- Prediction latency < 100ms
- System uptime > 99.9%
- Business impact measurable within 3 months

---

**Congratulations! You're now ready to deploy production ML models at Amazon scale!** 🚀

**Next Steps:**
1. Test your model with real data
2. Set up monitoring and alerting
3. Plan for model retraining
4. Scale based on business growth